In [1]:
import mysql.connector
import json, os, re
from datetime import datetime
import pandas as pd
import re

def get_json_paths(root_folder):
    json_paths = []
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        if os.path.isfile(folder_path) and folder_path.endswith(".json"):
            json_paths.append(folder_path)
    return json_paths

def load_all_records(json_paths):
    all_records = {}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
                records = data.get("records", [])
                name = " ".join(records[0].get("value", {}).get("amc_name", "").split(" ")[:2])
                if name not in all_records:
                    all_records[name] = records
                else:
                    name += " 2"
                    all_records[name] = records
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records

def load_all_records_mydata(json_paths):
    all_records = {}
    for path in json_paths:
        try:
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)  # The whole dict is the data
                sample_scheme = next(iter(data.values()))
                amc_name = sample_scheme.get("amc_name", "").strip()
                name = " ".join(amc_name.split(" ")[:2])
                
                if name not in all_records:
                    all_records[name] = data
                else:
                    name += " 2"
                    all_records[name] = data
        except Exception as e:
            print(f"Error reading {path}: {e}")
    return all_records

def null_if_empty(value):
    if value in ("", None):
        return None
    return value
with open("id_resolver_map.json", "r", encoding="utf-8") as f:
    id_maps = json.load(f)

def resolve_id(name, id_map_type):
    if not name:
        return None
    name = name.lower()
    for pattern, id_value in id_maps.get(id_map_type, {}).items():
        if re.search(pattern, name):
            return id_value
    return None

In [ ]:
# #myddata
# # conn = mysql.connector.connect(
# #     host="localhost",   
# #     user="root",            
# #     password="1234", 
# #     database="data_db",  
# #     port=3306,               
# #     charset="utf8mb4"
# # )

# # conn = mysql.connector.connect(
# #     host="172.22.225.155",
# #     user="cog_mf",
# #     password="bnYwFChjLAV2Z%9E",
# #     database="cog_mf",
# #     port=3306,
# #     charset='utf8mb4'
# # )

# # conn = mysql.connector.connect(
# #     host="localhost",   
# #     user="root",            
# #     password="1234", 
# #     database="cog_updated_db",  
# #     port=3306,               
# #     charset="utf8mb4"
# # )


# cursor = conn.cursor()

# def insert_common_fund_data(cursor, details, curr_time, amc_month, data_from,monthly_aaum_date):
#     amc_id = resolve_id(details.get("amc_name", ""), "amc_id_map")
#     fund_id = resolve_id(details.get("mutual_fund_name", ""), "fund_id_map")

#     keys = [
#         "amc_id", "fund_id", "entered_time", "amc_for_month", "data_from", "amc_name", "main_scheme_name", "fund_name",
#         "benchmark_index", "monthly_aaum_date", "monthly_aaum_value", "scheme_launch_date",
#         "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple",
#         "entry_load", "exit_load",
#         "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity",
#         "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay",
#         "mod_duration", "port_turnover_ratio", "r_squared_ratio", "roe_ratio", "sharpe", "sortino_ratio",
#         "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"
#     ]

#     metrics = details.get("metrics", {})
#     load = details.get("load", {})

#     values = [
#         amc_id,
#         fund_id,
#         null_if_empty(curr_time),
#         null_if_empty(amc_month),
#         null_if_empty(data_from),
#         null_if_empty(details.get("amc_name", "")),
#         null_if_empty(details.get("main_scheme_name", "")),
#         null_if_empty(details.get("mutual_fund_name", "")),
#         null_if_empty(details.get("benchmark_index", "")),
#         null_if_empty(monthly_aaum_date),
#         null_if_empty(details.get("monthly_aaum_value", "")),
#         null_if_empty(details.get("scheme_launch_date", "")),
#         null_if_empty(details.get("min_addl_amt", "")),
#         null_if_empty(details.get("min_addl_amt_multiple", "")),
#         null_if_empty(details.get("min_amt", "")),
#         null_if_empty(details.get("min_amt_multiple", "")),
#         null_if_empty(load.get("entry_load", "")),
#         null_if_empty(load.get("exit_load", "")),
#     ] + [null_if_empty(metrics.get(k, "")) for k in keys[18:]]

#     query = f"""
#         INSERT INTO mf_common_fund_data ({', '.join(keys)})
#         VALUES ({', '.join(['%s'] * len(keys))})
#     """

#     try:
#         cursor.execute(query, values)
#         return cursor.lastrowid, amc_id, fund_id
#     except Exception as e:
#         print("Error inserting common fund data:")
#         print("Scheme Name:", details.get("main_scheme_name"))
#         print("Exception:", e)
#         return None, None, None

# def insert_fund_manager_data(cursor, mutual_fund_id, amc_id, fund_id, details, curr_time, amc_month, data_from):
#     if not isinstance(details.get("fund_manager"), list):
#         return
#     for manager in details["fund_manager"]:
#         query = """
#             INSERT INTO mf_common_fund_manager_data
#             (mutual_fund_id, amc_id, fund_id, entered_time, amc_for_month, data_from,
#              amc_name, fund_name, main_scheme_name, name, qualification, managing_fund_since, total_exp)
#             VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
#         """
#         values = [
#             mutual_fund_id,
#             amc_id,
#             fund_id,
#             null_if_empty(curr_time),
#             null_if_empty(amc_month),
#             null_if_empty(data_from),
#             null_if_empty(details.get("amc_name", "")),
#             null_if_empty(details.get("mutual_fund_name", "")),
#             null_if_empty(details.get("main_scheme_name", "")),
#             null_if_empty(manager.get("name", "")),
#             null_if_empty(manager.get("qualification", "")),
#             null_if_empty(manager.get("managing_fund_since", "")),
#             null_if_empty(manager.get("total_exp", ""))
#         ]
#         try:
#             cursor.execute(query, values)
#         except Exception as e:
#             print(f"Error inserting fund manager: {e}")

# base_dir = os.path.join(os.path.dirname(os.getcwd()), "sql_learn", "mar25")
# json_paths = get_json_paths(base_dir)
# combined_records = load_all_records_mydata(json_paths)
# # curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
# # amc_month = "MAR25"
# # data_from = "mydata"
# # monthly_aaum_date = "20250321|March 2025|31032025"

# # for amc_name, schemes in combined_records.items():
# #     print(f"Doing: {amc_name}")
# #     for scheme_name, details in schemes.items():
# #         try:
# #             mutual_fund_id, amc_id, fund_id = insert_common_fund_data(cursor, details, curr_time, amc_month, data_from,monthly_aaum_date)
# #             if mutual_fund_id:
# #                 insert_fund_manager_data(cursor, mutual_fund_id, amc_id, fund_id, details, curr_time, amc_month, data_from)
# #             else:
# #                 print(f"Skipped Data Insertion for Scheme {scheme_name}")
# #         except Exception as e:
# #             print(f"Error inserting scheme '{scheme_name}' under AMC '{amc_name}': {e}")
# #     print(f"Done {amc_name}")

# # conn.commit()
# # cursor.close()
# # conn.close()
# # print("Working!!")


In [ ]:
#fink
conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)

# conn = mysql.connector.connect(
#     host="localhost",   
#     user="root",            
#     password="1234", 
#     database="cog_updated_db",  
#     port=3306,               
#     charset="utf8mb4"
# )

# conn = mysql.connector.connect(
#     host="localhost",   
#     user="root",            
#     password="1234", 
#     database="data_db",  
#     port=3306,               
#     charset="utf8mb4"
# )

cursor = conn.cursor()

def insert_common_fund_data(cursor, details, curr_time, amc_month, data_from):
    amc_id = resolve_id(details.get("amc_name", ""), "amc_id_map")
    fund_id = resolve_id(details.get("mutual_fund_name", ""), "fund_id_map")

    keys = [
        "amc_id", "fund_id", "entered_time", "amc_for_month", "data_from", "amc_name", "main_scheme_name", "fund_name",
        "benchmark_index", "monthly_aaum_date", "monthly_aaum_value", "scheme_launch_date",
        "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple",
        "entry_load", "exit_load",
        "alpha", "arithmetic_mean_ratio", "average_div_yld", "average_pb", "average_pe", "avg_maturity",
        "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay",
        "mod_duration", "port_turnover_ratio", "r_squared_ratio", "roe_ratio", "sharpe", "sortino_ratio",
        "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"
    ]
    
    entry_load = next((l.get("comment") for l in details.get("load", []) if l.get("type") == "entry_load"), "")
    exit_load = next((l.get("comment") for l in details.get("load", []) if l.get("type") == "exit_load"), "")


    values = [
        amc_id,
        fund_id,
        null_if_empty(curr_time),
        null_if_empty(amc_month),
        null_if_empty(data_from),
        null_if_empty(details.get("amc_name", "")),
        null_if_empty(details.get("main_scheme_name", "")),
        null_if_empty(details.get("mutual_fund_name", "")),
        null_if_empty(", ".join(details.get("benchmark_index", [])) if isinstance(details.get("benchmark_index"), list) else details.get("benchmark_index", "")),
        null_if_empty(details.get("monthly_aaum_date", "")),
        null_if_empty(details.get("monthly_aaum_value", "")),
        null_if_empty(details.get("scheme_launch_date", "")),
        null_if_empty(details.get("min_addl_amt", "")),
        null_if_empty(details.get("min_addl_amt_multiple", "")),
        null_if_empty(details.get("min_amt", "")),
        null_if_empty(details.get("min_amt_multiple", "")),
        null_if_empty(entry_load),
        null_if_empty(exit_load)
    ] + [
        null_if_empty(next((m.get("value") for m in details.get("metrics", []) if m.get("name") == key), ""))
        for key in keys[18:]
    ]

    query = f"""
        INSERT INTO mf_common_fund_data ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """
    # query = f"""
    #     INSERT INTO mf_keny_alldata ({', '.join(keys)})
    #     VALUES ({', '.join(['%s'] * len(keys))})
    # """
    
    try:
        cursor.execute(query, values)
        return cursor.lastrowid, amc_id, fund_id
    except Exception as e:
        print("Error inserting common fund data:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Exception:", e)
        return None, None, None

def insert_fund_manager_data(cursor, mutual_fund_id, amc_id, fund_id, details, curr_time, amc_month, data_from):
    if not isinstance(details.get("fund_manager"), list):
        return
    for manager in details["fund_manager"]:
        query = """
            INSERT INTO mf_common_fund_manager_data
            (mutual_fund_id, amc_id, fund_id, entered_time, amc_for_month, data_from,
             amc_name, fund_name, main_scheme_name, name, qualification, managing_fund_since, total_exp)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        values = [
            mutual_fund_id,
            amc_id,
            fund_id,
            null_if_empty(curr_time),
            null_if_empty(amc_month),
            null_if_empty(data_from),
            null_if_empty(details.get("amc_name", "")),
            null_if_empty(details.get("mutual_fund_name", "")),
            null_if_empty(details.get("main_scheme_name", "")),
            null_if_empty(manager.get("name", "")),
            null_if_empty(manager.get("qualification", "")),
            null_if_empty(manager.get("managing_fund_since", "")),
            null_if_empty(manager.get("total_exp", ""))
        ]
        try:
            cursor.execute(query, values)
        except Exception as e:
            print(f"Error inserting fund manager: {e}")

# Main execution
base_dir = os.path.join(os.path.dirname(os.getcwd()), "sql_learn", "json","APR25")
json_paths = get_json_paths(base_dir)
combined_records = load_all_records(json_paths)
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "APR25"
data_from = "mydata"

for amc_name, records in combined_records.items():
    print(f"Inserting AMC: {amc_name}")
    for record in records:
        try:
            details = record["value"]
            mutual_fund_id, amc_id, fund_id = insert_common_fund_data(cursor, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_manager_data(cursor, mutual_fund_id, amc_id, fund_id, details, curr_time, amc_month, data_from)
            else:
                print(f"Skipped: {details.get('main_scheme_name')}")
        except Exception as e:
            print(f"Error processing scheme under AMC '{amc_name}': {e}")

conn.commit()
cursor.close()
conn.close()
print("Finkstein data insertion completed.")

In [3]:
#mf_keny_alldata new

conn = mysql.connector.connect(
    host="172.22.225.155",
    user="cog_mf",
    password="bnYwFChjLAV2Z%9E",
    database="cog_mf",
    port=3306,
    charset='utf8mb4'
)
cursor = conn.cursor()

def insert_complete_fund_data(cursor, details, curr_time, amc_month, data_from):
    amc_id = resolve_id(details.get("amc_name", ""), "amc_id_map")
    fund_id = resolve_id(details.get("mutual_fund_name", ""), "fund_id_map")

    keys = [
        "amc_id", "fund_id", "entered_time", "amc_for_month", "amc_name", "main_scheme_name", "fund_name",
        "benchmark_index", "monthly_aaum_date", "monthly_aaum_value", "scheme_launch_date",
        "min_addl_amt", "min_addl_amt_multiple", "min_amt", "min_amt_multiple",
        "entry_load", "exit_load",
        "alpha", "arithmetic_mean_ratio", "average_div_yield", "average_pb", "average_pe", "avg_maturity",
        "beta", "correlation_ratio", "downside_deviation", "information_ratio", "macaulay",
        "mod_duration", "port_turnover_ratio", "r_squared_ratio", "roe_ratio", "sharpe", "sortino_ratio",
        "std_dev", "tracking_error", "treynor_ratio", "upside_deviation", "ytm"
    ]

    # Load extraction
    entry_load = next((l.get("comment") for l in details.get("load", []) if l.get("type") == "entry_load"), "")
    exit_load = next((l.get("comment") for l in details.get("load", []) if l.get("type") == "exit_load"), "")

    # Raw values before null cleanup
    raw_values = [
        amc_id,
        fund_id,
        curr_time,
        amc_month,
        details.get("amc_name", ""),
        details.get("main_scheme_name", ""),
        details.get("mutual_fund_name", ""),
        ", ".join(details.get("benchmark_index", [])) if isinstance(details.get("benchmark_index"), list) else details.get("benchmark_index", ""),
        details.get("monthly_aaum_date", ""),
        details.get("monthly_aaum_value", ""),
        details.get("scheme_launch_date", ""),
        details.get("min_addl_amt", ""),
        details.get("min_addl_amt_multiple", ""),
        details.get("min_amt", ""),
        details.get("min_amt_multiple", ""),
        entry_load,
        exit_load
    ] + [
        next((m.get("value") for m in details.get("metrics", []) if m.get("name") == key), "")
        for key in keys[17:]
    ]

    # Apply null_if_empty just once
    values = [null_if_empty(v) for v in raw_values]

    query = f"""
        INSERT INTO mf_keny_alldata ({', '.join(keys)})
        VALUES ({', '.join(['%s'] * len(keys))})
    """

    try:
        cursor.execute(query, values)
        return cursor.lastrowid, amc_id, fund_id
    except Exception as e:
        print("Error inserting into complete fund data:")
        print("Scheme Name:", details.get("main_scheme_name"))
        print("Exception:", e)
        return None, None, None
    
def insert_fund_manager_data(cursor, details, curr_time, amc_month):
    if not isinstance(details.get("fund_manager"), list):
        return

    for manager in details["fund_manager"]:
        keys = [
            "entered_time", "amc_for_month",
            "main_scheme_name", "name", "managing_fund_since", "qualification", "total_experience",
            "entered_user"  # Optional, if tracking users
        ]

        raw_values = [
            curr_time,
            amc_month,
            details.get("main_scheme_name", ""),
            manager.get("name", ""),
            manager.get("managing_fund_since", ""),
            manager.get("qualification", ""),
            manager.get("total_exp", ""),  # mapped to `total_experience`
            "system"  # or fetch from user/session if dynamic
        ]

        values = [null_if_empty(v) for v in raw_values]

        query = f"""
            INSERT INTO mf_keny_fund_manager ({', '.join(keys)})
            VALUES ({', '.join(['%s'] * len(keys))})
        """

        try:
            cursor.execute(query, values)
        except Exception as e:
            print("Error inserting fund manager data:")
            print("AMC:", details.get("amc_name", ""))
            print("Fund Manager Name:", manager.get("name", ""))
            print("Exception:", e)

# Main execution
base_dir = os.path.join(os.path.dirname(os.getcwd()), "sql_learn", "json","hello")
json_paths = get_json_paths(base_dir)
combined_records = load_all_records(json_paths)
curr_time = datetime.now().strftime("%Y-%m-%d %H:%M")
amc_month = "APR25"
data_from = "mydata"

for amc_name, records in combined_records.items():
    print(f"Inserting AMC: {amc_name}")
    for record in records:
        try:
            details = record["value"]
            mutual_fund_id, amc_id, fund_id = insert_complete_fund_data(cursor, details, curr_time, amc_month, data_from)
            if mutual_fund_id:
                insert_fund_manager_data(cursor,details, curr_time, amc_month)
            else:
                print(f"Skipped: {details.get('main_scheme_name')}")
        except Exception as e:
            print(f"Error processing scheme under AMC '{amc_name}': {e}")

conn.commit()
cursor.close()
conn.close()
print("Kenytable data insertion completed.")


Inserting AMC: LIC Mutual
Kenytable data insertion completed.


In [ ]:
# CREATE DATABASE data_db;

# CREATE TABLE mf_common_fund_data (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     amc_id INT,
#     fund_id INT,
#     entered_time DATETIME,
#     amc_for_month VARCHAR(200),
#     data_from VARCHAR(200),
#     amc_name TEXT,
#     main_scheme_name TEXT,
#     fund_name TEXT,
#     benchmark_index TEXT,
#     monthly_aaum_date TEXT,
#     monthly_aaum_value TEXT,
#     scheme_launch_date TEXT,
#     min_addl_amt TEXT,
#     min_addl_amt_multiple TEXT,
#     min_amt TEXT,
#     min_amt_multiple TEXT,
#     entry_load TEXT,
#     exit_load TEXT,
#     alpha VARCHAR(120),
#     arithmetic_mean_ratio VARCHAR(120),
#     average_div_yld VARCHAR(120),
#     average_pb VARCHAR(120),
#     average_pe VARCHAR(120),
#     avg_maturity VARCHAR(120),
#     beta VARCHAR(120),
#     correlation_ratio VARCHAR(120),
#     downside_deviation VARCHAR(120),
#     information_ratio VARCHAR(120),
#     macaulay VARCHAR(120),
#     mod_duration VARCHAR(120),
#     port_turnover_ratio VARCHAR(120),
#     r_squared_ratio VARCHAR(120),
#     roe_ratio VARCHAR(120),
#     sharpe VARCHAR(120),
#     sortino_ratio VARCHAR(120),
#     std_dev VARCHAR(120),
#     tracking_error VARCHAR(120),
#     treynor_ratio VARCHAR(120),
#     upside_deviation VARCHAR(120),
#     ytm VARCHAR(120)
# );

# CREATE TABLE mf_common_fund_manager_data (
#     id INT AUTO_INCREMENT PRIMARY KEY,
#     mutual_fund_id INT,
#     amc_id INT,
#     fund_id INT,
#     entered_time DATETIME,
#     amc_for_month VARCHAR(200),
#     data_from VARCHAR(200),
#     amc_name TEXT,
#     fund_name TEXT,
#     main_scheme_name TEXT,
#     name TEXT,
#     qualification TEXT,
#     managing_fund_since TEXT,
#     total_exp TEXT,
#     CONSTRAINT fk_mutual_fund_id FOREIGN KEY (mutual_fund_id) REFERENCES mf_common_fund_data(id) ON DELETE CASCADE
# );
